<a href="https://colab.research.google.com/github/TarnaiMark/AdvStat/blob/main/halstat_survival.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercises

In [ ]:
!pip install lifelines scikit-survival

In [ ]:
import numpy as np
import pandas as pd
import lifelines as lf
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

## 0.) Pre-processing

Load the <i><a href="https://scikit-survival.readthedocs.io/en/stable/api/generated/sksurv.datasets.load_veterans_lung_cancer.html">Veteran's Lung Cancer</a></i> dataset from <code>scikit-survival</code>! Note that it returns a <code>Pandas</code> DataFrame of the features, and a structured array containing the survival times, and censoring status.

In [ ]:
from sksurv.datasets import load_veterans_lung_cancer
df,y = load_veterans_lung_cancer()

Add the survival time and censoring status to the DataFrame as new columns!

Take a look at the features! What kind of features do you see?

Use one-hot-encoding on the categorical features!

Hint: [<code>pd.get_dummies(drop_first=True)</code>](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html)

Scale the the numeric features (except for the survival time) to have zero mean and unit variance!

Hint: [`sklearn.preprocessing.scale`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html)

In [ ]:
from sklearn.preprocessing import scale

## 1.) Kaplan-Meier estimator

Stratify the population according to some rule! This could be based on categories or numerical features. Calculate the Kaplan-Meier estimator (<code style="font-size:9px;"><a href="https://lifelines.readthedocs.io/en/latest/fitters/univariate/KaplanMeierFitter.html">lifelines.fitters.kaplan_meier_fitter.KaplanMeierFitter</a></code>) for each group and plot the results. Do the plots seem to show any differences?

Perform a logrank test to see whether these splits are meaningful. You can use <code style="font-size:9px;"><a href="https://lifelines.readthedocs.io/en/latest/lifelines.statistics.html#lifelines.statistics.logrank_test">lifelines.statistics.logrank_test</a></code>.





Heed the advice in the lifelines docs and repeat the testing with likelihood ratio test. Are the splits still meaningful? Fit a Cox proportional hazards model on a binary indicator of group membership. If the corresponding coefficient is confirmed by the likelihood ratio test to be non-zero, that means the grouping is meaningful.

In [ ]:
from lifelines.fitters.coxph_fitter import CoxPHFitter

## 2.)The Cox Proportional Hazards Model

Fit the [Cox proportional hazards model](https://lifelines.readthedocs.io/en/latest/fitters/regression/CoxPHFitter.html) to the preprocessed dataset! Plot the weights! According to them, which feature is the most influential?"

Is there any feature where the hypothesis for a zero coefficient cannot be discarded? Look at the Wald-test results!

Could these results be due to the data having some time-varying effect? Check the assumptions by using a 95% confidence interval and plotting the scaled Schoenfeld residuals!

Heed **Advice 2** and stratify the model. After stratification, each stratum will have its own CPH model that will hopefully satisfy the proportional hazards assumption.

Hint: Use [`pd.qcut`](https://pandas.pydata.org/docs/reference/api/pandas.qcut.html) to bin any numeric features according to the median (2-quantile).

## 3.) Concordance index
Unfortunetly Uno's IPC-weighted c-index is not implemented in lifelines, so we have to use the [scikit-survival implementation](https://scikit-survival.readthedocs.io/en/stable/api/generated/sksurv.metrics.concordance_index_ipcw.html). This also requires [`sksurv.utils.Surv`](https://scikit-survival.readthedocs.io/en/stable/api/generated/sksurv.util.Surv.html) to create the structured arrays, which are then passed on to the metric.

Calculate the c-index, for various truncation times in a 5-fold cross-validation scheme for both the stratified and unstratified models! What do you observe?

In [ ]:
from sksurv.util import Surv
from sksurv.metrics import concordance_index_ipcw
from sklearn.model_selection import KFold

def concordance_index_ipcw_5fold_scorer(model,df,tau,**kwargs):

What do you think are the reasons for the results?